In [1]:
# Dependencies
import pandas as pd
#load data
wage_df = pd.read_csv("Resources/Minimum Wage Data.csv", encoding="ISO-8859-1")
state_df = pd.read_csv("Resources/state_lat_long.csv", encoding="ISO-8859-1")

In [2]:
#Fillna for High Value
wage_df["High.Value"].fillna(0.00, inplace = True) 


In [3]:
#cleaning up raw wage DF and up[dating column name
wage_df_clean = wage_df[["Year", "State", "High.Value", "CPI.Average", "High.2018"]].copy()
wage_df_clean.rename(columns = {"Year": "Year", "State": "State",'High.Value':'Minimum Wage', 'CPI.Average':'CPI Average', 
                              'High.2018':'Wage equivalent in 2018'}, inplace = True) 
wage_df_clean["Minimum Wage"] = wage_df_clean["Minimum Wage"].map("{:,.2f}".format)


In [4]:
#Data dictionary to add fereal minimum wage and equivalent 2018 dollar value

fed_wage_eqvt_2018 ={1968:11.55,1969:10.95,1970:10.36,1971:9.93,1972:9.62,1973:9.05,1974:10.19,1975:9.81,1976:10.16,1977:9.54,1978:10.21,1979:10.04,1980:9.45,1981:9.26,1982:8.72,1983:8.45,1984:8.10,1985:7.82,1986:7.68,1987:7.41,1988:7.11,1989:6.79,1990:7.3,1991:7.84,1992:7.61,1993:7.39,1994:7.25,1995:7.01,1996:7.61,1997:8.06,1998:7.94,1999:7.77,2000:7.51,2001:7.31,2002:7.19,2003:7.03,2004:6.85,2005:6.62,2006:6.42,2007:7.09,2008:7.64,2009:8.49,2010:8.35,2011:8.10,2012:7.93,2013:7.82,2014:7.69,2015:7.68,2016:7.59,2017:7.43}
fed_wage_def ={1968:1.60,1969:1.60,1970:1.60,1971:1.60,1972:1.60,1973:1.60,1974:2.00,1975:2.10,1976:2.30,1977:2.30,1978:2.65,1979:2.90,1980:3.10,1981:3.35,1982:3.35,1983:3.35,1984:3.35,1985:3.35,1986:3.35,1987:3.35,1988:3.35,1989:3.35,1990:3.80,1991:4.25,1992:4.25,1993:4.25,1994:4.25,1995:4.25,1996:4.75,1997:5.15,1998:5.15,1999:5.15,2000:5.15,2001:5.15,2002:5.15,2003:5.15,2004:5.15,2005:5.15,2006:5.15,2007:5.85,2008:6.55,2009:7.25,2010:7.25,2011:7.25,2012:7.25,2013:7.25,2014:7.25,2015:7.25,2016:7.25,2017:7.25}

In [5]:
#updating wage df with new values using above dictionaries
for i, wage in enumerate(wage_df_clean["Minimum Wage"]):
    if (wage == "0.00"):
       wage_df_clean.loc[i,'Minimum Wage'] = fed_wage_def[wage_df_clean.loc[i,"Year"]]
       wage_df_clean.loc[i,'Wage equivalent in 2018'] = fed_wage_eqvt_2018[wage_df_clean.loc[i,"Year"]]

In [6]:
#Making API call to create a new poverty rate dataframe for each state between 1998 to 2017
import requests
pvt_data = []
url = "https://api.census.gov/data/timeseries/poverty/saipe?get=NAME,SAEPOVRTALL_PT&for=state&time="
for i in range(1998,2018):
    pvt_response = requests.get(url + str(i)).json()
   
    for y in range(1,len(pvt_response)):
         pvt_data.append([pvt_response[y][2], pvt_response[y][0], pvt_response[y][1]] )
pvt_df = pd.DataFrame(pvt_data)


In [7]:
#merging State df (latitude longitude) with wage df
wage_df_clean = pd.merge(state_df, wage_df_clean, how='outer', on='State')

In [8]:
#merge pvt df with updated wage df and renaming columns
pvt_df.rename(columns = {0:'Year', 1:'State',2:"Poverty Rate"}, inplace = True)
pvt_df['Year'] = pvt_df['Year'].astype(int)
final_data = pd.merge(wage_df_clean, pvt_df, how='outer', on=['Year', 'State'])

#remove few states that doesn't have poverty data
final_data = final_data[final_data['State'] != 'Guam']
final_data = final_data[final_data['State'] != 'Federal (FLSA)']
final_data = final_data[final_data['State'] != 'Puerto Rico']
final_data = final_data[final_data['State'] != 'U.S. Virgin Islands']
final_data["CPI Average"] = final_data["CPI Average"].map("{:,.2f}".format)

In [10]:
final_data.to_csv("Output/minimum_wage_and_poverty_data.csv", index=False, header=True)